In [2]:
import pathlib
import os
import random
import time
import functools
from collections import OrderedDict
import requests
import sys
from time import sleep

### Function basics

In [3]:
def func(x, y, z):
    print(x, y, z)

func(1, 2, 3)
func(x=1, y=2, z=3)

1 2 3
1 2 3


In [7]:
def func(x='abc', y=42, z=None):
    print(x, y, z)


func()
func('xyz')
# func(, 43)
func(y=43)
# func('xyz', 43, (0,1))

abc 42 None
xyz 42 None
abc 43 None


In [ ]:
class BaseModelForm(BaseForm):
    def __init__(self, data=None, files=None, auto_id='id_%s', prefix=None,
                 initial=None, error_class=ErrorList, label_suffix=None,
                 empty_permitted=False, instance=None, use_required_attribute=None,
                 renderer=None):
        ...

In [11]:
def func(*args):
    print(args)

func('xyz')
func('xyz', 43, (0,1))
# func(y=43)

('xyz',)
('xyz', 43, (0, 1))


In [10]:
def func(*args, **kwargs):
    print('args:', args, ', kwargs:', kwargs)


func()
func('xyz')
func('xyz', 43, (0,1))
func(y=43)

args: () , kwargs: {}
args: ('xyz',) , kwargs: {}
args: ('xyz', 43, (0, 1)) , kwargs: {}
args: () , kwargs: {'y': 43}


### Everything is an object, so is a function

In [15]:
def func(x='abc', y=42, z=None):
    print(x, y, z)

func1 = func
type(func1)

function

In [16]:
print(id(func1), id(func))
func1()

140306123544464 140306123544464
abc 42 None


<img src=https://miro.medium.com/max/700/1*n0MVfelkQrJ_zBym4b3CSA.png>

<img src=https://i.imgur.com/aye8Juy.png width=700>


In [17]:
def func(f):
    f()

def foo():
    print('Hello from foo')

def bar():
    print('Hello from bar')

func(foo)
func(bar)

Hello from foo
Hello from bar


In [22]:
def foo():
    print('Hello from foo')

def func(f):
    return f

result = func(foo)
# type(result)
result()
foo()

Hello from foo
Hello from foo


In [24]:
def hello_factory():
    def internal():
        print('Hello from internal')
    return internal

func = hello_factory()
func()
# internal()



Hello from internal


In [26]:
def transparent_factory(f):
    def internal():
        f()       
    return internal

foo_transparent = transparent_factory(foo)
foo_transparent()

foo()

Hello from foo
Hello from foo


In [27]:
def hello_factory_2(f):
    def internal():
        print(f">>> Is about to call '{f.__name__}'\n")
        f()
        print(f"\n>>> Just called '{f.__name__}'")
        
    return internal

foo_verbosed = hello_factory_2(foo)
foo_verbosed()

foo()

>>> Is about to call 'foo'

Hello from foo

>>> Just called 'foo'
Hello from foo


In [29]:
def profile(f):
    def internal():
        start = time.time()
        f()
        finish = time.time()
        print(f'Elapsed time: {finish - start}s')
    return internal

foo_profiled = profile(foo)
foo_profiled()

foo()


Hello from foo
Elapsed time: 0.000247955322265625s
Hello from foo


### Decorators

#### D. is a function that adds some auxilary functionality, typically not changing original behaviour.

<img src=https://d33wubrfki0l68.cloudfront.net/12c8a296cc396d418b5407a4a4c6f9fd7d85f597/e8a54/wp-content/uploads/2018/06/python-decorator.png>

In [30]:
def profile(f):
    def internal():
        start = time.time()
        f()
        print(f'Elapsed time ({f.__name__}): {time.time() - start}s')
    return internal

foo_profiled = profile(foo)
foo_profiled()


Hello from foo
Elapsed time (foo): 0.00012826919555664062s


In [34]:
def fetch_url():
    res = requests.get('https://google.com')
    print('Content: \t', res.content[:100])

In [35]:
fetch_url_profiled = profile(fetch_url)
fetch_url_profiled()

Content: 	 b'<!doctype html><html itemscope="" itemtype="http://schema.org/WebPage" lang="uk"><head><meta content'
Elapsed time (fetch_url): 0.8245177268981934s


In [36]:
def fetch_url(url):
    res = requests.get(url)
    print(f'\nContent for "{url}":\t ', res.content[:100])

def profile(f):
    def internal(url):
        start = time.time()
        f(url)
        print(f'Elapsed time ({f.__name__}): {time.time() - start}s')
    return internal

fetch_url_profiled = profile(fetch_url)
fetch_url_profiled('https://google.com')
fetch_url_profiled('https://ithillel.ua')
fetch_url_profiled('https://lms.ithillel.ua')


Content for "https://google.com":	  b'<!doctype html><html itemscope="" itemtype="http://schema.org/WebPage" lang="uk"><head><meta content'
Elapsed time (fetch_url): 0.9915337562561035s

Content for "https://ithillel.ua":	  b'\xef\xbb\xbf\n\n\n\n\n\t\n\n\n\t\t\t\t\n\t\n\t\n\t\n\n\n\n\n        \n    \t\t\t\t\n\n        \n\t<!DOCTYPE html>\n\t<html lang="ru">\n\t<head>\n\n\t'
Elapsed time (fetch_url): 0.8337750434875488s

Content for "https://lms.ithillel.ua":	  b'<!DOCTYPE html>\n<html lang="ru">\n  <head>\n    <meta charset="utf-8"/>\n    <meta name="viewport" cont'
Elapsed time (fetch_url): 0.2980961799621582s


In [52]:

def profile(f):
    def internal(*args):
        start = time.time()
        f(*args)
        print(f'Elapsed time ({f.__name__}): {time.time() - start}s')
    return internal

fetch_url = profile(fetch_url)
fetch_url('https://ithillel.ua')


Content for "https://ithillel.ua":	  b'\xef\xbb\xbf\n\n\n\n\n\t\n\n\n\t\t\t\t\n\t\n\t\n\t\n\n\n\n\n        \n    \t\t\t\t\n\n        \n\t<!DOCTYPE html>\n\t<html lang="ru">\n\t<head>\n\n\t'
Elapsed time (fetch_url): 0.3529641628265381s


In [38]:
def fetch_url(url, first_n=None):
    res = requests.get(url)
    print(f'\nContent for "{url}":\t ', res.content[:first_n] if first_n else res.content)

def profile(f):
    def internal(*args, **kwargs):
        start = time.time()
        f(*args, **kwargs)
        print(f'Elapsed time ({f.__name__}): {time.time() - start}s')
    return internal

fetch_url = profile(fetch_url)
fetch_url('https://ithillel.ua', first_n=42)
fetch_url('https://ithillel.ua', first_n=100)
fetch_url('https://ithillel.ua')


Content for "https://ithillel.ua":	  b'\xef\xbb\xbf\n\n\n\n\n\t\n\n\n\t\t\t\t\n\t\n\t\n\t\n\n\n\n\n        \n    \t\t'
Elapsed time (fetch_url): 1.0209336280822754s

Content for "https://ithillel.ua":	  b'\xef\xbb\xbf\n\n\n\n\n\t\n\n\n\t\t\t\t\n\t\n\t\n\t\n\n\n\n\n        \n    \t\t\t\t\n\n        \n\t<!DOCTYPE html>\n\t<html lang="ru">\n\t<head>\n\n\t'
Elapsed time (fetch_url): 0.5450992584228516s

Content for "https://ithillel.ua":	  b'\xef\xbb\xbf\n\n\n\n\n\t\n\n\n\t\t\t\t\n\t\n\t\n\t\n\n\n\n\n        \n    \t\t\t\t\n\n        \n\t<!DOCTYPE html>\n\t<html lang="ru">\n\t<head>\n\n\t\t<link rel="preload" href="https://assets.ithillel.ua/fonts/solomon_sans_normal.woff2" as="font" type="font/woff2" crossorigin="anonymous">\n<link rel="preload" href="https://assets.ithillel.ua/fonts/solomon_sans_semibold.woff2" as="font" type="font/woff2" crossorigin="anonymous">\n<link rel="preload" href="https://assets.ithillel.ua/fonts/solomon_sans_bold.woff2" as="font" type="font/woff2" crossorigin=

In [39]:
def fetch_url(url, first_n=100):
    res = requests.get(url)
    return res.content[:first_n] if first_n else res.content

fetch_url = profile(fetch_url)
with open('/tmp/content.html', 'w+') as f:
    f.write(str(fetch_url('https://ithillel.ua', first_n=42)))

# cat /tmp/content.html


Elapsed time (fetch_url): 0.23036694526672363s


In [40]:
def profile(f):
    def internal(*args, **kwargs):
        start = time.time()
        result = f(*args, **kwargs)
        print(f'Elapsed time for function {f.__name__} with params {args}, {kwargs}: {time.time() - start}ms')
        return result
    return internal

fetch_url = profile(fetch_url)
with open('/tmp/content.html', 'w+') as f:
    f.write(str(fetch_url('https://ithillel.ua', first_n=42)))


Elapsed time (fetch_url): 0.1851346492767334s
Elapsed time for function internal with params ('https://ithillel.ua',), {'first_n': 42}: 0.18528103828430176ms


In [79]:
@profile # -> fetch_url = profile(fetch_url)
def fetch_url(url, first_n=100):
    res = requests.get(url)
    return res.content[:first_n] if first_n else res.content
    
print(fetch_url('https://ithillel.ua', first_n=42))
print(fetch_url('https://google.com'))

Elapsed time for function fetch_url with params ('https://ithillel.ua',), {'first_n': 42}: 0.1624441146850586ms
b'\xef\xbb\xbf\n\n\n\n\n\t\n\n\n\t\t\t\t\n\t\n\t\n\t\n\n\n\n\n        \n    \t\t'
Elapsed time for function fetch_url with params ('https://google.com',), {}: 0.7504243850708008ms
b'<!doctype html><html itemscope="" itemtype="http://schema.org/WebPage" lang="uk"><head><meta content'


### We need to go deeper

In [87]:
def profile(f):
    def internal(*args, **kwargs):
        start = time.time()
        result = f(*args, **kwargs)
        print(f'Elapsed time for function {f.__name__} with params {args}, {kwargs}: {time.time() - start}ms')
        return result
    return internal


@profile
def fetch_url(url, first_n=100):
    """Fetch a given url"""
    res = requests.get(url)
    return res.content[:first_n] if first_n else res.content

help(fetch_url)

Help on function internal in module __main__:

internal(*args, **kwargs)



In [88]:
def profile(f):
    @functools.wraps(f)
    def internal(*args, **kwargs):
        start = time.time()
        result = f(*args, **kwargs)
        print(f'Elapsed time for function {f.__name__} with params {args}, {kwargs}: {time.time() - start}ms')
        return result
    return internal


@profile
def fetch_url(url, first_n=100):
    """Fetch a given url"""
    res = requests.get(url)
    return res.content[:first_n] if first_n else res.content


help(fetch_url)

Help on function fetch_url in module __main__:

fetch_url(url, first_n=100)
    Fetch a given url



In [91]:
def profile(msg='Elapsed time'):
    def internal(f):
        @functools.wraps(f)
        def deco(*args, **kwargs):
            start = time.time()
            result = f(*args, **kwargs)
            print(msg, f'({f.__name__}): {time.time() - start}s')
            return result
        return deco
    return internal

In [94]:
@profile(msg='Прошло времени')
def fetch_url(url, first_n=100):
    """Fetch a given url"""
    res = requests.get(url)
    return res.content[:first_n] if first_n else res.content

fetch_url('https://google.com')
    
@profile(msg='Elapsed time')
def fetch_url(url, first_n=100):
    """Fetch a given url"""
    res = requests.get(url)
    return res.content[:first_n] if first_n else res.content

fetch_url('https://google.com')

Прошло времени (fetch_url): 0.7934281826019287s
Elapsed time (fetch_url): 0.7412421703338623s


b'<!doctype html><html itemscope="" itemtype="http://schema.org/WebPage" lang="uk"><head><meta content'

In [93]:
@profile('Elapsed time') # -> profile_ = profile('Time spent')
                         # -> foo5 = profile_(foo5)
def foo():
    """Help for foo"""
    return 42

help(foo)
print("RESULT: ", foo())

Help on function foo in module __main__:

foo()
    Help for foo

Elapsed time (foo): 2.86102294921875e-06s
RESULT:  42


### Template

#### 2-level decorator

In [117]:
def profile(f):
    @functools.wraps(f)
    def deco(*args, **kwargs):
        start = time.time()
        result = f(*args, **kwargs)
        print('Elapsed time', f'({f.__name__}): {time.time() - start}s')
        return result
    return deco

@profile
def foo(): ...

#### 3-level decorator

In [47]:
def profile(msg='Elapsed time', file=sys.stdout):
    def internal(f):
        @functools.wraps(f)
        def deco(*args, **kwargs):
            start = time.time()
            result = f(*args, **kwargs)
            print(msg, f'({f.__name__}): {time.time() - start}s', file=file)
            return result
        return deco
    return internal

@profile()
def foo(): ...


### Real life examples

In [95]:
# 0 1 2 3 4 5 6 7 8
# 0 1 1 2 3 5 8 13 ...
def fibo(n):
    """Super inefficient fibo function"""
    if n < 2:
        return n
    else:
        return fibo(n-1) + fibo(n-2)
   
fibo(10)

55

In [97]:
@profile()
def fibo(n):
    """Super inefficient fibo function"""
    if n < 2:
        return n
    else:
        return fibo(n-1) + fibo(n-2)
   
fibo(10)

Elapsed time (fibo): 1.1920928955078125e-06s
Elapsed time (fibo): 1.430511474609375e-06s
Elapsed time (fibo): 0.0003650188446044922s
Elapsed time (fibo): 4.76837158203125e-07s
Elapsed time (fibo): 0.0004184246063232422s
Elapsed time (fibo): 4.76837158203125e-07s
Elapsed time (fibo): 7.152557373046875e-07s
Elapsed time (fibo): 4.3392181396484375e-05s
Elapsed time (fibo): 0.0005056858062744141s
Elapsed time (fibo): 2.384185791015625e-07s
Elapsed time (fibo): 4.76837158203125e-07s
Elapsed time (fibo): 4.5299530029296875e-05s
Elapsed time (fibo): 2.384185791015625e-07s
Elapsed time (fibo): 9.512901306152344e-05s
Elapsed time (fibo): 0.0006499290466308594s
Elapsed time (fibo): 2.384185791015625e-07s
Elapsed time (fibo): 7.152557373046875e-07s
Elapsed time (fibo): 4.506111145019531e-05s
Elapsed time (fibo): 4.76837158203125e-07s
Elapsed time (fibo): 8.940696716308594e-05s
Elapsed time (fibo): 2.384185791015625e-07s
Elapsed time (fibo): 7.152557373046875e-07s
Elapsed time (fibo): 4.5537948608

55

#### Recursion support

In [121]:
def profile(msg="Elapsed time for function"):
    def internal(f):
        @functools.wraps(f)
        def deco(*args, **kwargs):
            start = time.time()
            deco._num_call += 1
            result = f(*args, **kwargs)
            deco._num_call -= 1
            
            if deco._num_call == 0:
                print(msg, f'{f.__name__}: {time.time() - start}s')
            return result
        
        deco._num_call = 0
        return deco
    
    return internal

@profile()
def fibo(n):
    """Super inefficient fibo function"""
    if n < 2:
        return n
    else:
        return fibo(n-1) + fibo(n-2)
   
fibo(10)

Elapsed time for function fibo: 0.0001461505889892578s


55

#### Exponential backoff

In [98]:
def repeate(max_repeat=10):
    def internal(f):
        @functools.wraps(f)
        def repeater(*args, **kwargs):
            while repeater._num_repeats <= max_repeat:
                try:
                    return f(*args, **kwargs)
                except Exception as ex:
                    if repeater._num_repeats == max_repeat:
                        raise
                    else:
                        print(f'Failed after {repeater._num_repeats+1} times, trying again after {2**repeater._num_repeats} sec...')
                        sleep(2**repeater._num_repeats)
                        repeater._num_repeats += 1
                
        repeater._num_repeats = 0
        return repeater
    return internal


@repeate(max_repeat=4)
# @repeate()
# @repeate # note the difference
def connect_to_server(*args):
    print('Trying to connect: ', *args)
    if sum(random.choices([0, 1], [0.8, 0.2])) == 0:
        raise RuntimeError('Failed to connect')
    print('SUCCESS!')

connect_to_server('google.com')


Trying to connect:  google.com
Failed after 1 times, trying again after 1 sec...
Trying to connect:  google.com
Failed after 2 times, trying again after 2 sec...
Trying to connect:  google.com
Failed after 3 times, trying again after 4 sec...
Trying to connect:  google.com
SUCCESS!


#### Cache

In [101]:
def cache(f):
    
    @functools.wraps(f)
    def deco(*args):
        
        if args in deco._cache:
            return deco._cache[args]
        
        result = f(*args)
        
        deco._cache[args] = result
        
        return result
    
    deco._cache = {}
        
    return deco


In [112]:
def cache(max_limit=64):
    def internal(f):
        @functools.wraps(f)
        def deco(*args):
            if args in deco._cache:
                # Перемещение только что использованного элемента в конец списка на удаление
                deco._cache.move_to_end(args, last=True)
                return deco._cache[args]
            result = f(*args)
            # Удаление из словаря значения, если в нем больше чем задано max_limit. Для Домашнего задания
            if len(deco._cache) >= max_limit:
                 # удаление первого элемента
                deco._cache.popitem(last=False)
            deco._cache[args] = result
            return result
        deco._cache = OrderedDict()
        return deco
    return internal

In [118]:
@profile(msg='Elapsed time')
@cache(max_limit=5)
# @profile(msg='Elapsed time')
# @cache
def fetch_url(url, first_n=100):
    """Fetch a given url"""
    res = requests.get(url)
    return res.content[:first_n] if first_n else res.content

fetch_url('https://google.com')
fetch_url('https://google.com')
fetch_url('https://google.com')
fetch_url('https://ithillel.ua')
fetch_url('https://dou.ua')
fetch_url('https://ain.ua')
fetch_url('https://youtube.com')
fetch_url('https://reddit.com')
fetch_url._cache

Elapsed time (fetch_url): 1.0068199634552002s
Elapsed time (fetch_url): 4.0531158447265625e-06s
Elapsed time (fetch_url): 3.0994415283203125e-06s
Elapsed time (fetch_url): 0.20612883567810059s
Elapsed time (fetch_url): 0.23122024536132812s
Elapsed time (fetch_url): 0.18753814697265625s
Elapsed time (fetch_url): 0.5199284553527832s
Elapsed time (fetch_url): 3.4398229122161865s


OrderedDict([(('https://ithillel.ua',),
              b'\xef\xbb\xbf\n\n\n\n\n\t\n\n\n\t\t\t\t\n\t\n\t\n\t\n\n\n\n\n        \n    \t\t\t\t\n\n        \n\t<!DOCTYPE html>\n\t<html lang="ru">\n\t<head>\n\n\t'),
             (('https://dou.ua',),
              b'<html>\r\n<head><title>403 Forbidden</title></head>\r\n<body>\r\n<center><h1>403 Forbidden</h1></center>\r\n'),
             (('https://ain.ua',),
              b'<html>\r\n<head><title>403 Forbidden</title></head>\r\n<body>\r\n<center><h1>403 Forbidden</h1></center>\r\n'),
             (('https://youtube.com',),
              b'<!DOCTYPE html><html style="font-size: 10px;font-family: Roboto, Arial, sans-serif;" lang="uk-UA" sy'),
             (('https://reddit.com',),
              b'\n    <!DOCTYPE html>\n    <html lang="en-US">\n      <head>\n        <script>\n    var __SUPPORTS_TIMING')])

<img src=https://i.imgur.com/IgBFGaR.png>

In [189]:
@profile()
@cache
def fibo(n):
    """Super inefficient fibo function"""
    if n < 2:
        return n
    else:
        return fibo(n-1) + fibo(n-2)

In [190]:
# for i in range(10000):
#     print(i, '->', fibo(i))
print(fibo(500))

Elapsed time for function fibo: 0.0016300678253173828s
139423224561697880139724382870407283950070256587697307264108962948325571622863290691557658876222521294125


In [114]:
# fibo._cache
# id(fibo._cache)

In [128]:
from functools import lru_cache

@lru_cache(maxsize=64)
def foo():
    print('foo')

### Links

1. Python Introduction: https://www.youtube.com/watch?v=5V7XG1mGiHc&list=PLlb7e2G7aSpTTNp7HBYzCBByaE1h54ruW
2. Теория по декораторам: http://bit.ly/2z5yatp: стр. 414
3. Примеры с урока: https://github.com/dbradul/python_course/blob/master/lesson_deco.ipynb
4. Exponential backoff: https://bit.ly/3nvOyeQ
5. Установка jupyter: https://www.youtube.com/watch?v=VIaur9G-0tc

### H/w

1. Дополнить декоратор сache поддержкой max_limit.
2. 